In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 23 07:17:44 2021

@author: ajith.masthand
"""

### Test-Control pairing function
def tc_pairing():
    ### Run Interactively
    global int_response
    int_response=interactive()
    
    inp_path=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Pairing - Inputs',index_col='input_parameter')
    global inp_par
    inp_par=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Pairing - Interactive',index_col='input_parameter')
    
    tc_input=pd.read_csv(inp_path['value']['input_data_path'])
    tc_input['IMS_ID']=tc_input['IMS_ID'].astype(str)
    print("\nData imported with below columns\n")
    print(list(tc_input.columns))
    
    ### Creating aggregate columns
    print("\nAggregate columns will be created for TRX, NRX, PDE, SAM using C3M, P3M, C6M, P6M, 12M")
    
    for var in ['TRX','NRX','PDE','SAM']:
        tc_input[var+'_C3M']=tc_input.loc[:,var+'_01':var+'_03'].sum(axis=1)
        tc_input[var+'_P3M']=tc_input.loc[:,var+'_04':var+'_06'].sum(axis=1)
        tc_input[var+'_C6M']=tc_input.loc[:,var+'_01':var+'_06'].sum(axis=1)
        tc_input[var+'_P6M']=tc_input.loc[:,var+'_07':var+'_12'].sum(axis=1)
        tc_input[var+'_12M']=tc_input.loc[:,var+'_01':var+'_12'].sum(axis=1)
    del var
    
    ### Universe Filter
    tc_input=analysis_univ(tc_input)
    
    ### Deciling
    tc_input=ntile(tc_input)
    
    ### Test Control Split
    test_data,control_data=tc_split(tc_input)
    
    ### Test Control Pairing
    
    ### Splitting Test data into chunks to have < 10K HCPs per chunk to avoid memory / sizing issues
    ### And creating Test Control pairs by processing multiple chunk level pairs
    ### The function includes cat_match, tol_limit, normalize and pairing functions
    tc_match,tc_pair,tc_dist=chunk_proc(tc_input,test_data,control_data)
    
    ### Type of pairing to be implemented
    ### The function matches the pairs based on One-One or Many-One mapping
    tc_pair,tc_dist=pair_type(tc_pair,tc_dist,test_data,control_data)
    
    ### Optional Iterations for Leftover / Unmapped Test / Control HCPs
    
    response = 'y'
    while (response.lower() == "y"):
        print("\nDo you wish to map leftover Test / Control HCPs? (Y/N)")
        response = input()
        if (response.lower() == "n"):
            print("\nFinal Test Control pairing Summary\n")
            print(tc_pair['T_IMS_ID'].unique().size," Test HCPs are matched with ",
                  tc_pair['C_IMS_ID'].unique().size," Control HCPs","\n")
            print(test_data['T_IMS_ID'].unique().size," Test HCPs and ",
                  control_data['C_IMS_ID'].unique().size," Control HCPs were created in Total")
            ### Generating trend charts for Test Control pairs
            #Creating Test Data, Control Data with out prefixes T_ or C_
            #Creating campaign exposed HCPs same as Test HCPs who are paired
            test_data_1=test_data.copy()
            control_data_1=control_data.copy()
            test_data_1.columns=test_data_1.columns.str[2:]
            control_data_1.columns=control_data.columns.str[2:]
            camp_hcp=pd.DataFrame(tc_pair.loc[:,'T_IMS_ID'])
            cross_master_data,vertical_master_data=lift_master(tc_pair,test_data_1,control_data_1,camp_hcp)
            trend_data_dict=pairing_trends(cross_master_data,vertical_master_data)
            ### Exporting Final results
            output(tc_input,tc_pair,inp_path,trend_data_dict)
        elif (response.lower() == "y"):
            int_response=interactive()
            inp_par=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Pairing - Interactive',index_col='input_parameter')
            tc_pair,test_data,control_data=remap(tc_pair,tc_input,test_data,control_data)
        else:
            print("\nInvalid Response. Please type Y or N")
    
    del response
    del inp_par,inp_path,int_response
    del tc_dist,tc_match
    ### Pairing Complete
     
    return tc_input,tc_pair,test_data,control_data
   
### TC pairing interactive function
def interactive():
    print("\nWould you like to run interactively (Y/N)?")
    response="NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        response=input()
        if (response.lower() == "n"):
            interactive="n"
            print("\nThe tool will be run with the inputs provided in 'Input Parametrs.xlsx' file")
            print("\nIs the 'Pairing - Interactive' sheet in the excel file updated with all the parameters (Y/N)")
            update="NA"
            while (update.lower() != "y"):
                update = input()
                if (update.lower() == "n"):
                    print("\nPlease update the sheet and type Y")
                elif (update.lower() == "y"):
                    print("\nParameters in 'Pairing - Interactive' sheet will be used")
                else:
                    print("\nInvalid Response. Please type Y or N")
        elif (response.lower() == "y"):
            interactive="y"
            print("\nThe tool will be run intractively\n")
        else:
            print("\nInvalid Response. Please type Y or N")
    return interactive

### TC pairing Analysis Universe
def analysis_univ(tc_input):
    tc_input=tc_input.copy()
    print("\n",tc_input.shape[0]," records are present in the data")
    print("\nDo you wish to filter analysis universe based on certain columns in the data? (Y/N)")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_1']
            print(response)
        
        if (response.lower() == "n"):
            print("\nAnalysis Universe is unaltered")
        elif (response.lower() == "y"):
            print(list(tc_input.columns))
            print("\nEnter the column name/s on which 0 value records need to be filtered out separated by a comma(,) and a space( )?")
            print("\nEg.: If you enter PDE_12M, NRX_12M records with both PDE_12M as 0 AND NRX_12M as 0 will be filtered out")
            response = "NA"
            while (np.invert(set(response).issubset(tc_input.columns))):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['columns_1']
                    print(response)
                
                response=response.split(", ")
                if (set(response).issubset(tc_input.columns)):
                    tc_input['univ_flag']=tc_input[response].sum(axis=1)
                    tc_input=tc_input.loc[tc_input['univ_flag']>0,:].drop(['univ_flag'],axis='columns')
                    print("\n",tc_input.shape[0]," records are present in the data after filtering")
                else:
                    print("\nInvalid Response. Please type correct column name/s")
            response="y"
        else:
            print("\nInvalid Response. Please type Y or N")
    return tc_input

### TC pairing ntiling
def ntile(tc_input):
    tc_input=tc_input.copy()
    print("\nDo you have a column for ntile(Decile, Quintile)? (Y/N)")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_2']
            print(response)
        
        if (response.lower() == "n"):
            print("\nDo you wish to create ntile? (Y/N)\n")
            response = "NA"
            while ((response.lower() != "y") & (response.lower() != "n")):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['y_n_3']
                    print(response)
                
                if (response.lower() == "n"):
                    print("\nProceeding with out ntiling")
                elif (response.lower() == "y"):
                    print(list(tc_input.columns))
                    print("\nEnter the name of the column on which ntiling has to be performed")
                    response = "NA"
                    while (sum(tc_input.columns == response) == 0):
                        if (int_response=="y"):
                            response=input()
                        else:
                            response=inp_par['value']['columns_3']
                            print(response)
                        
                        if (sum(tc_input.columns == response) > 0):
                            print("\nEnter value of n for ntiling (10 for deciling, 5 for quintiling etc..)")
                            response1=0
                            while (response1 < 1):
                                if (int_response=="y"):
                                    response1=int(input())
                                else:
                                    response1=int(inp_par['value']['number_3'])
                                    print(response1)
                                
                                if (response1 >= 1):
                                    print("\nntiling will be performed on ",response," with n = ",response1)
                                else:
                                    print("\nInvalid Response. Please type an integer value > 1")
                            
                            decile_data=tc_input[['IMS_ID',response]]
                            decile_data=decile_data.sort_values(response,ascending=False).reset_index(drop=True)
                            decile_data['cum_rx']=decile_data[response].cumsum()
                            rx_sum=decile_data[response].sum()
                            decile_data['decile']=response1-(((decile_data['cum_rx']/(rx_sum+0.9999))*response1).astype(int))
                            decile_data.loc[decile_data[response]==0,'decile']=0
                            
                            ## Dropping 'decile' column from original data if it's present to avoid duplicates
                            if (sum(tc_input.columns == 'decile') > 0):
                                tc_input=tc_input.drop(['decile'],axis='columns')
                            tc_input=pd.merge(tc_input,decile_data[['IMS_ID','decile']],on='IMS_ID',how='left')
                            del rx_sum, decile_data, response1
                            print("\nntiling complete based on column ",response)
                            print("\ndecile column has been created with decile / ntile information")
                        else:
                            print("\nInvalid Response. Please type the correct column name")
                    response='y'
                else:
                    print("\nInvalid Response. Please type Y or N")
            response='n'
        elif (response.lower() == "y"):
            print(list(tc_input.columns))
            print("\nEnter the name of the column which contains ntile")
            response = "NA"
            while (sum(tc_input.columns == response) == 0):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['columns_2']
                    print(response)
                
                if (sum(tc_input.columns == response) > 0):
                    print("\n",response," column will be used for decile")
                    tc_input['decile']=tc_input[response]
                    print("\ndecile column has been created with decile / ntile information")
                else:
                    print("\nInvalid Response. Please type the correct column name")
            response='y'
        else:
            print("\nInvalid Response. Please type Y or N")
    return tc_input


### TC pairing Test Control Split
def tc_split(tc_input):
    tc_input=tc_input.copy()
    print("\nDo you have a TC_Flag that indicates whether HCP is a Test or Control? (Y/N)\n")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_4']
            print(response)
        
        if (response.lower() == "n"):
            print("\nTest & Control split will be done using Stratified sampling approach")
            print("\nEnter % of HCPs you want as your Test Universe (Eg: If you want 80% of your universe to be Test HCPs and 20% to be Control HCPs, Enter 80)\n")
            response = 0
            while ((response < 1) | (response > 99)):
                if (int_response=="y"):
                    response = int(input())
                else:
                    response=int(inp_par['value']['number_4'])
                    print(response)
                
                if (1 <= response <= 99):
                    print(list(tc_input.columns))
                    print("\nEnter column name/s on which you wish to maintain similar distribution in Test/Control universes separated by a comma(,) and a space( )")
                    response1 = "NA"
                    while (np.invert(set(response1).issubset(tc_input.columns))):
                        if (int_response=="y"):
                            response1=input()
                        else:
                            response1=inp_par['value']['columns_4']
                            print(response1)
                        response1=response1.split(", ")
                        if (set(response1).issubset(tc_input.columns)):
                            print("\nStratified Sampling will be performed with ",response,"% as Test HCPs and maintaining similar distribution of ",response1," in Test/Control Universe")
                            test_data, control_data = train_test_split(tc_input, train_size=response/100, 
                                                                       random_state=45, 
                                                                       stratify=tc_input[response1])
                        else:
                            print("\nInvalid Response. Please type the correct column name/s")
                    del response1
                else:
                    print("\nInvalid Response. Please type a value between 1 - 99")
            response='n'
        elif (response.lower() == "y"):
            print(list(tc_input.columns))
            print("\nEnter the name of the column that indicates whether HCP is a Test or Control? (The column must have T or C as it's distinct values')")
            response="NA"
            while (sum(tc_input.columns == response) == 0):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['columns_4']
                    print(response)
                if (sum(tc_input.columns == response) > 0):
                    print(response," column will be used as Test Control Flag")
                    test_data=tc_input.loc[tc_input[response].str.upper()=='T',:]
                    control_data=tc_input.loc[tc_input[response].str.upper()=='C',:]
                else:
                    print("\nInvalid Response. Please type the correct column name")
            response='y'
        else:
            print("Invalid Response. Please type Y or N")
    print("\nTest & Control data is separated and columns in Test & Control data are prefixed with T_ & C_ respectively")
    test_data.columns="T_"+test_data.columns
    control_data.columns="C_"+control_data.columns
    return test_data,control_data

### TC pairing Matching on Categorical data
def cat_match(tc_input,test_data,control_data):
    tc_input=tc_input.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    print("\nDo you wish to match Test & Control pairs based on certain categorical data first? (Y/N)")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_5']
            print(response)
        if (response.lower() == "n"):
            test_data['tmp']=1
            control_data['tmp']=1
            tc_match=pd.merge(test_data,control_data,on='tmp',how='inner').drop(['tmp'],axis='columns')
            if (tc_match.isnull().sum(axis=1).sum() > 0):
                print("\nNull values present in the data are replaced with 0")
                tc_match.fillna(0)
            print("\n",tc_match.T_IMS_ID.unique().size," Test HCPs are matched with ",
                  tc_match.C_IMS_ID.unique().size," Control HCPs")
            print("\nProceeding to Tolerance Matching")
        elif (response.lower() == "y"):
            print("\nDisplaying column names for reference\n")
            print(list(tc_input.columns))
            print("\nEnter name/s of the column/s on which you wish to have an exact match separated by a comma(,) and a space( )")
            response = "NA"
            while (np.invert(set(response).issubset(tc_input.columns))):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['columns_5']
                    print(response)
                response=response.split(', ')
                if (set(response).issubset(tc_input.columns)):
                    tc_match=pd.merge(test_data,control_data,how='inner',
                                      left_on=["T_" + x for x in response],
                                      right_on=["C_" + x for x in response])
                    if (tc_match.isnull().sum(axis=1).sum() > 0):
                        print("\nNull values present in the data are replaced with 0")
                        tc_match.fillna(0)
                else:
                    print("\nInvalid Response. Please type correct name/s")
                print("\nBefore matching on Categorical data")
                print("\n",test_data.T_IMS_ID.unique().size," Test HCPs are matched with ",
                      control_data.C_IMS_ID.unique().size," Control HCPs")
                print("\nAfter matching on Categorical data")
                print("\n",tc_match.T_IMS_ID.unique().size," Test HCPs are matched with ",
                      tc_match.C_IMS_ID.unique().size," Control HCPs")
            response="y"
        else:
            print("\nInvalid Response. Please type Y or N")
    return tc_match

### TC pairing Tolerance limit application
def tol_limit(tc_input,tc_match):
    tc_input=tc_input.copy()
    tc_match=tc_match.copy()
    print("\nDo you wish to set tolerance limits on certain metrics of Control HCPs? (Y/N)")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_6']
            print(response)
        if (response.lower() == "n"):
            print("\nProceeding to standardize numerical data")
            break
        elif (response.lower() == "y"):
            print("\nDisplaying column names for reference\n")
            print(list(tc_input.columns))
            print("\nEnter name/s of the column/s on which you wish set tolerance limits separated by a comma(,) and a space( )")
            response = "NA"
            while (np.invert(set(response).issubset(tc_input.columns))):
                if (int_response=="y"):
                    response=input()
                else:
                    response=inp_par['value']['columns_6']
                    print(response)
                response=response.split(', ')
                if (set(response).issubset(tc_input.columns)):
                    print("\nEnter Tolerance limits for each of these metrics in the same order separated by a comma(,) and a space( )")
                    print("\nEnter values between 0 and 1. Eg. 0 is exact match, 0.1 is +/- 10%, 0.7 is +/- 70% etc..")
                    response1=list(range(1,10000))
                    while (len(response1)!=len(response)):
                        if (int_response=="y"):
                            response1=str(input())
                        else:
                            response1=str(inp_par['value']['number_6'])
                            print(response1)
                        response1=response1.split(", ")
                        response1=[float(x) for x in response1]
                        if (len(response1)==len(response)):
                            counter=0
                            print("\nBefore Tolerance application")
                            print("\n",tc_match.T_IMS_ID.unique().size," Test HCPs are matched with ",
                                  tc_match.C_IMS_ID.unique().size," Control HCPs")
                            for var in response:
                                limit=response1[counter]
                                tc_match=tc_match.loc[tc_match['C_'+var].between((1-limit)*tc_match['T_'+var],
                                                                                 (1+limit)*tc_match['T_'+var],
                                                                                 inclusive=True),:]
                                counter=counter+1
                            print("\nAfter Tolerance application")
                            print("\n",tc_match.T_IMS_ID.unique().size," Test HCPs are matched with ",
                                  tc_match.C_IMS_ID.unique().size," Control HCPs")
                            del counter, var, limit
                        else:
                            print("\nInvalid Response. Please type correct order of values")
                else:
                    print("\nInvalid Response. Please type correct name/s")
                del response1
            response="y"
        else:
            print("\nInvalid Response. Please type Y or N")
    return tc_match

### TC pairing Data Standardization
def normalize(tc_input,tc_match):
    tc_input=tc_input.copy()
    tc_match=tc_match.copy()
    print("\nDisplaying column names for reference\n")
    print(list(tc_input.columns))
    print("\nEnter column name/s on which you wish to calculate distance")
    response = "NA"
    while (np.invert(set(response).issubset(tc_input.columns))):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['columns_7']
            print(response)
        response=response.split(', ')
        response.append('IMS_ID')
        if (set(response).issubset(tc_input.columns)):
            response1=pd.concat([pd.Series(["T_" + x for x in response]),pd.Series(["C_" + x for x in response])])
            tc_dist=tc_match[response1]
            print("\nChoose type of standardization \n1 - Z Score Normalization \n2 - Min Max Normalization \n3 - No Normalization \n(Enter 1 / 2 / 3)")
            response2=0
            while ((response2 != 1) & (response2 != 2) & (response2 != 3)):
                if (int_response=="y"):
                    response2=int(input())
                else:
                    response2=int(inp_par['value']['number_7'])
                    print(response2)
                
                if (response2==1):
                    for var in tc_dist.columns:
                        if ((var != 'T_IMS_ID') & (var != 'C_IMS_ID')):
                            tc_dist[var]=(tc_dist[var] - tc_dist[var].mean())/tc_dist[var].std(ddof=0)
                    print("\nVariables Normalized with Z-Score Normalization")
                    del var
                elif (response2==2):
                    for var in tc_dist.columns:
                        if ((var != 'T_IMS_ID') & (var != 'C_IMS_ID')):
                            mini=tc_dist[var].min()
                            maxi=tc_dist[var].max()
                            tc_dist[var]=(tc_dist[var] - mini)/(maxi - mini)
                            del mini,maxi
                    print("\nVariables Normalized with Min-Max Normalization")
                    del var
                elif (response2==3):
                    print("\nProceeding with out normalization")
                else:
                    print ("\nInvalid Response. Please type 1 or 2 or 3")
            
            del response1,response2
        else:
            print("\nInvalid Response. Please type correct name/s")
    return tc_dist

### TC pairing Creating pairs
def pairing(tc_dist,test_data,control_data):
    tc_dist=tc_dist.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    print("\nChoose Algorithm for Test & Control matching \n1 - Euclidean Distance\n2 - Propensity Score\nEnter 1 / 2")
    response=0
    while ((response!=1) & (response!=2)):
        if (int_response=="y"):
            response=int(input())
        else:
            response=int(inp_par['value']['number_8'])
            print(response)
        
        if (response==1):
            weights,wt_columns=weightages(tc_dist)
            tc_dist['euc_dist']=0
            counter=0
            for var in (wt_columns):
                tc_dist['euc_dist']=tc_dist['euc_dist']+((tc_dist['T_'+var]-tc_dist['C_'+var])**2)*weights[counter]
                counter=counter+1
            del var
            tc_dist=tc_dist.sort_values(['T_IMS_ID','euc_dist'],ascending = True)
            tc_pair=tc_dist.drop_duplicates(subset=['T_IMS_ID'],keep='first')
            tc_pair=tc_pair[['T_IMS_ID','C_IMS_ID']]
            print("\nChunk wise Test Control pairs are created using Euclidean distance method\n")
            print("\nChunk level mapping summary\n")
            print(tc_pair['T_IMS_ID'].unique().size," Test HCPs are matched with ",
                  tc_pair['C_IMS_ID'].unique().size," Control HCPs")
            print(test_data['T_IMS_ID'].unique().size," Test HCPs and ",
                  control_data['C_IMS_ID'].unique().size," Control HCPs were present originally","\n")
        elif (response==2):
            print("\nPropensity Score matching algorithm is WIP Please select 1 - Euclidean Distance")
            response=0
        else:
            print("\nInvalid Response. Please type 1 / 2")
    return tc_pair,tc_dist

### Weightages to distance calculation variables
def weightages(tc_dist):
    tc_dist=tc_dist.copy()
    wt_columns=list(tc_dist.columns.str[2:].unique())
    wt_columns.remove('IMS_ID')
    print("\nDo you wish to use weightages while calculating Euclidean Distance? (Y/N)")
    print("If 'N' is selected, equal weightages will be assigned")
    response = "NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['y_n_7']
            print(response)
        if (response.lower() == "n"):
            print("\nEqual weightages will be used")
            weightage=1/(len(wt_columns))
            weights=[weightage for i in range(tc_dist.columns.str[2:].unique().size-1)]
            print(weights)
        elif (response.lower() == "y"):
            print("\nEnter weights for the variables selected in the same order as below separated by a comma(,) and a space( )")
            print("\nEnter values between 0 and 1.")
            print(wt_columns)
            response1=list(range(1,10000))
            while ((len(response1)!=len(wt_columns)) | (sum(response1)<0.999) | (sum(response1)>1.001)):
                if (int_response=="y"):
                    response1=str(input())
                else:
                    response1=str(inp_par['value']['number_9'])
                    print(response1)
                response1=response1.split(", ")
                response1=[float(x) for x in response1]
                if ((len(response1)==len(wt_columns)) & (sum(response1)>0.999) & (sum(response1)<1.001)):
                    weights=response1
                else:
                    print("\nInvalid Response. Please type correct order of values")
        else:
            print("\nInvalid Response. Please type Y or N")
    return weights,wt_columns

### TC pairing chunk processing
def chunk_proc(tc_input,test_data,control_data):
    tc_input=tc_input.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    
    number_of_chunks=math.ceil(test_data.shape[0]/5000)
    print("\nStarting Split process with chunks: ", str(number_of_chunks))
    for id, df_i in  enumerate(np.array_split(test_data, number_of_chunks)):
        print("\nPairing chunk number ",id+1)
        #### Exact match on Categorical data
        tc_match=cat_match(tc_input,df_i,control_data)

        #### Tolerance limits on continuous variables for control HCPs
        ##### Eliminating pairs by setting tolerance limits on Control metric w.r.t Test metric
        ##### Eg. (1 - limit*Test_Metric) <= Control_Metric <= (1 + limit*Test_Metric)
        tc_match=tol_limit(tc_input,tc_match)
        
        #### Standardizing the data using Z-Score Normalization / Min-Max Normalization
        tc_dist=normalize(tc_input,tc_match)
        
        #### Using Euclidean Distance / Propensity score to create pairs
        ##### Selecting Control which has rank = 1
        locals()['tc_pair'+str(id)],locals()['tc_dist'+str(id)]=pairing(tc_dist,df_i,control_data)
        if(id==0):
            tc_pair=locals()['tc_pair'+str(id)]
            tc_dist=locals()['tc_dist'+str(id)]
        else:
            tc_pair=pd.concat([tc_pair,locals()['tc_pair'+str(id)]],axis=0,ignore_index=True)
            tc_dist=pd.concat([tc_dist,locals()['tc_dist'+str(id)]],axis=0,ignore_index=True)
        del locals()['tc_pair'+str(id)],locals()['tc_dist'+str(id)] , df_i,id                       
    print("\nPairing completed with Split process")
    return tc_match,tc_pair,tc_dist

### TC pairing type of pairing
def pair_type(tc_pair,tc_dist,test_data,control_data):
    tc_pair=tc_pair.copy()
    tc_dist=tc_dist.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    
    print("\nChoose one of the below methods for type of Test Control pairs\n")
    print("1a - Many - One: One Test mapped to One Control & One Control can be mapped to many Tests")
    print("1b - One - One: One Test mapped to One Control & One Control mapped to one Test")
    print("\nEnter label number of your choice (Eg. 1a)")
    
    match_type="NA"
    while (np.invert(match_type in ['1a','1b'])):
        
        if (int_response=="y"):
            match_type=input()
        else:
            match_type=inp_par['value']['choice_1']
            print(match_type)
        if((match_type=='1a') | (match_type=='1b')):
            if(match_type=='1b'):
                tc_dist['control_count'] = tc_dist['T_IMS_ID'].map(tc_dist['T_IMS_ID'].value_counts())
                tc_dist=tc_dist.sort_values(['control_count','euc_dist','T_IMS_ID','C_IMS_ID'],ascending = True).drop(['control_count'],axis='columns')
                tc_pair=tc_dist.drop_duplicates(subset=['C_IMS_ID'],keep='first').drop_duplicates(subset=['T_IMS_ID'],keep='first')
                tc_pair=tc_pair[['T_IMS_ID','C_IMS_ID']]
            
            print("\nTest Control pairs are created using Euclidean distance method by collating different chunks\n")
            print("\nIntermediate mapping summary\n")
            print(tc_pair['T_IMS_ID'].unique().size," Test HCPs are matched with ",
                  tc_pair['C_IMS_ID'].unique().size," Control HCPs")
            print(test_data['T_IMS_ID'].unique().size," Test HCPs and ",
                  control_data['C_IMS_ID'].unique().size," Control HCPs were present originally","\n")
        else:
            print("\nEnter appropriate label number of your choice (Eg. 1a) ")
    return tc_pair,tc_dist



### TC pairing Remapping unmapped HCPs

def remap(tc_pair,tc_input,test_data,control_data):
    tc_pair=tc_pair.copy()
    tc_input=tc_input.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    print("\nRemapping Unmapped Test / Control HCPs through below methods\n")
    print("\nChoose from the below 3 choices if Test & Control universe are originally given separately\n")
    print("1a - Leftover Test HCPs as new Test, Complete Control Universe as new Control")
    print("1b - Leftover Test HCPs as new Test, Leftover Control Universe as new control")
    print("1c - Leftover Test HCPs as new Test, Matched Control Universe as new control")
    print("\nChoose from the below 3 choices if Test & Control universe are originally created through sampling\n")
    print("2a - Combine Leftover Test & Leftover Control, ReCreate new Test & new Control from this universe")
    print("2b - Combine Leftover Test & Leftover Control, ReCreate new Test & new Control from this universe, Add Matched Control Universe to the new controls")
    print("2c - Combine Leftover Test & Leftover Control as new Test, Matched Control Universe as new Control")
    print("\nEnter label number of your choice (Eg. 1a)")
    response="NA"
    while (np.invert(response in ['1a','1b','1c','2a','2b','2c'])):
        response=input()
        if (response=='1a'):
            new_test_data=test_data.loc[~(test_data['T_IMS_ID'].isin(tc_pair['T_IMS_ID'])),:]
            new_control_data=control_data.copy()
        elif (response=='1b'):
            new_test_data=test_data.loc[~(test_data['T_IMS_ID'].isin(tc_pair['T_IMS_ID'])),:]
            new_control_data=control_data.loc[~(control_data['C_IMS_ID'].isin(tc_pair['C_IMS_ID'])),:]
        elif (response=='1c'):
            new_test_data=test_data.loc[~(test_data['T_IMS_ID'].isin(tc_pair['T_IMS_ID'])),:]
            new_control_data=control_data.loc[control_data['C_IMS_ID'].isin(tc_pair['C_IMS_ID']),:]
        elif (response=='2a'):
            new_tc_input=tc_input.loc[~(tc_input['IMS_ID'].isin(tc_pair['T_IMS_ID'].append(tc_pair['C_IMS_ID']))),:]
            new_test_data,new_control_data=tc_split(new_tc_input)
        elif (response=='2b'):
            new_tc_input=tc_input.loc[~(tc_input['IMS_ID'].isin(tc_pair['T_IMS_ID'].append(tc_pair['C_IMS_ID']))),:]
            new_test_data,new_control_data=tc_split(new_tc_input)
            new_control_data=new_control_data.append(control_data.loc[control_data['C_IMS_ID'].isin(tc_pair['C_IMS_ID']),:])
        elif (response=='2c'):
            new_test_data=tc_input.loc[~(tc_input['IMS_ID'].isin(tc_pair['T_IMS_ID'].append(tc_pair['C_IMS_ID']))),:]
            new_test_data.columns="T_"+new_test_data.columns
            new_control_data=control_data.loc[control_data['C_IMS_ID'].isin(tc_pair['C_IMS_ID']),:]
        else:
            print("Invalid response. Please type label from the list of choices")
    print("\nNew Test & new Control Universe have been created based on selected choice")
    
    ### Splitting Test data into chunks to have < 10K HCPs per chunk to avoid memory / sizing issues
    ### And creating Test Control pairs by processing multiple chunk level pairs
    ### The function includes cat_match, tol_limit, normalize and pairing functions
    new_tc_match,new_tc_pair,new_tc_dist=chunk_proc(tc_input,new_test_data,new_control_data)
    
    ### Type of pairing to be implemented
    ### The function matches the pairs based on One-One or Many-One mapping
    new_tc_pair,new_tc_dist=pair_type(new_tc_pair,new_tc_dist,new_test_data,new_control_data)
    
    ### Updating test data & control data by appending matched TC pair Test/Control data with new Test/Control data 
    mat_test_data=test_data.loc[test_data['T_IMS_ID'].isin(tc_pair['T_IMS_ID']),:]
    mat_control_data=control_data.loc[control_data['C_IMS_ID'].isin(tc_pair['C_IMS_ID']),:]
    upd_test_data=mat_test_data.append(new_test_data).drop_duplicates()
    upd_control_data=mat_control_data.append(new_control_data).drop_duplicates()
    
    ### Updating Test Control pairs
    upd_tc_pair=tc_pair.append(new_tc_pair).drop_duplicates()
    print("\nUpdated mapping summary\n")
    print(upd_tc_pair['T_IMS_ID'].unique().size," Test HCPs are matched with ",
          upd_tc_pair['C_IMS_ID'].unique().size," Control HCPs")
    print(upd_test_data['T_IMS_ID'].unique().size," Test HCPs and ",
          upd_control_data['C_IMS_ID'].unique().size," Control HCPs were given to this iteration","\n")
    
    return upd_tc_pair,upd_test_data,upd_control_data
    
### Goodness of fit trend charts for Test Control pairs
def pairing_trends(cross_master_data,vertical_master_data):
    cross_master_data=cross_master_data.copy()
    vertical_master_data=vertical_master_data.copy()
    
    print("\nEnter oldest and latest month numbers to be used for trend charts")
    print("Eg. 12 for oldest month number, 01 for latest month number")
    print("\nEnter month number for oldest month")
    if (int_response=="y"):
        old_month=input()
    else:
        old_month=inp_par['value']['number_10']
        print(old_month)
    
    print("\nEnter month number for latest month")
    if (int_response=="y"):
        lat_month=input()
    else:
        lat_month=inp_par['value']['number_11']
        print(lat_month)
    
    print("\nEnter metrics you wish to compare trends b/w Test & Control separated by a comma(,) and a space( )")
    print("Enter prefixes of the columns already present in the data")
    print("Eg.: NRX, PDE")
    response = ["NULLVALUE"]
    while (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) > 0):
        if (int_response=="y"):
            response=input()
        else:
            response=inp_par['value']['columns_8']
            print(response)
        response=response.split(', ')
        response=[x.upper() for x in response]
        if (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) == 0):
            trend_data_dict={}
            for var in response:
                trend_data_1=pd.DataFrame(cross_master_data.loc[:,'T_' + var + '_' + lat_month:'T_' + var + '_' + old_month].mean(axis=0).reset_index())
                trend_data_1['index']=trend_data_1['index'].str.replace("T_"+var+"_","").astype(int)
                trend_data_1.columns=['month',"T_"+var]
                
                trend_data_2=pd.DataFrame(cross_master_data.loc[:,'C_' + var + '_' + lat_month:'C_' + var + '_' + old_month].mean(axis=0).reset_index())
                trend_data_2['index']=trend_data_2['index'].str.replace("C_"+var+"_","").astype(int)
                trend_data_2.columns=['month',"C_"+var]
                
                trend_data=pd.merge(trend_data_1,trend_data_2,on='month',how='outer').sort_values(by='month',ascending=False)
                del trend_data_1,trend_data_2
                key_name=var+'_trend'
                trend_data_dict[key_name] = copy.deepcopy(trend_data)
        else:
            print("\nInvalid Response. Please type correct prefixes of columns")
    return trend_data_dict

### TC pairing Writing Output to Excel file
def output(tc_input,tc_pair,inp_path,trend_data_dict):
    test_data_out=tc_input.loc[tc_input.IMS_ID.isin(tc_pair['T_IMS_ID'].unique()),:]
    control_data_out=tc_input.loc[tc_input.IMS_ID.isin(tc_pair['C_IMS_ID'].unique()),:]
    with pd.ExcelWriter(inp_path['value']['output_data_path']) as writer:  
        tc_pair.to_excel(writer, sheet_name='TC_Pair',index=False)
        test_data_out.to_excel(writer, sheet_name='Test_Data',index=False)
        control_data_out.to_excel(writer, sheet_name='Control_Data',index=False)
        for key in trend_data_dict:
            trend_data_dict[key].to_excel(writer, sheet_name=key,index=False)
    
    print("\nFinal results exported to \n",inp_path['value']['output_data_path'])
    print("\nTest & Control pairing complete")


### Run lift calculation in continuation with Test-Control pairing
def run_lift(tc_pair,test_data,control_data):
    tc_pair=tc_pair.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    test_data.columns=test_data.columns.str[2:]
    control_data.columns=control_data.columns.str[2:]
    print("\nWould you like to run Lift calculation with the output of Test-Control matching (Y/N)?")
    response="NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        response=input()
        if (response.lower() == "n"):
            print("Thank You!!!")
            lift_out=pd.DataFrame(np.nan,index=[0], columns=['0'])
        elif (response.lower() == "y"):
            lift_calc(tc_pair,test_data,control_data)
        else:
            print("\nInvalid Response. Please type Y or N")
    

### Lift calculation Function
def lift_calc(tc_pair,test_data,control_data):
    tc_pair=tc_pair.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    
    ### File paths
    global lift_path
    lift_path=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Lift - Inputs',index_col='input_parameter')
    
    ### Finalising inputs
    tc_pair,test_data,control_data,camp_hcp=lift_input(tc_pair,test_data,control_data)
    
    ### Run Interactively
    global lift_int
    lift_int=lift_interactive()
    global lift_par
    lift_par=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Lift - Interactive',index_col='input_parameter')
    
    ### Pre Period & post Period
    pre_period,post_period=lift_periods(test_data)
    
    ### Master Data
    cross_master_data,vertical_master_data=lift_master(tc_pair,test_data,control_data,camp_hcp)
    
    ### Impact Calculaion
    camp_impact,camp_lift,emmeans,ancova_summary=lift_impact(cross_master_data,pre_period,post_period,vertical_master_data)
    
    ### Visualization of Lift analysis
    trend_data_dict=lift_trends(cross_master_data,vertical_master_data)
    
    ### Exporting Lift analysis output
    lift_output(trend_data_dict,emmeans,ancova_summary)
    
    print("\nImpact of campaign on ",lift_param," is observed to be ",camp_impact,lift_param," per month per HCP")
    print("With a lift of ",camp_lift*100,"%")
    print("\nLift calculation analysis complete")
    

#### Lift Inputs function
def lift_input(tc_pair,test_data,control_data):
    if (lift_calc_indep=="n"):
        tc_pair=tc_pair.copy()
        test_data=test_data.copy()
        control_data=control_data.copy()
        print("\nTest-Control pairs, Test Data, Control Data will be used from output of Test-Control pairing")
        print("\nChoose from below for list of campaign reached hcps (1/2/3)")
        print("\n1 - Consider Test HCPs from Test-Control pairs data as campaign reached HCPs")
        print("2 - Consider campaign reached HCPs list from the location given in 'Lift - Inputs' sheet of 'Input Parameters.xlsx' file ")
        print("3 - Enter location of .csv file which contains campaign reached HCPs list")
        response="NA"
        while (np.invert(response in ['1','2','3'])):
            response=input()
            if (response == "1"):
                camp_hcp=tc_pair.drop_duplicates(subset='T_IMS_ID').drop(['C_IMS_ID'],axis='columns')
            elif (response == "2"):
                print("\nIs the 'Lift - Inputs' sheet in the excel file updated with the location of campaign reached HCPs (Y/N)")
                update="NA"
                while (update.lower() != "y"):
                    update = input()
                    if (update.lower() == "n"):
                        print("\nPlease update the sheet and type Y")
                    elif (update.lower() == "y"):
                        print("\nLocations in 'Lift - Inputs' sheet will be used")
                    else:
                        print("\nInvalid Response. Please type Y or N")
                lift_path=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Lift - Inputs',index_col='input_parameter')
                camp_hcp=pd.read_csv(lift_path['value']['reached_ims_path'])
                camp_hcp['T_IMS_ID']=camp_hcp['T_IMS_ID'].astype(str)
            elif (response == "3"):
                print("\nInput Location of csv file")
                loc=input()
                camp_hcp=pd.read_csv(loc)
                camp_hcp['T_IMS_ID']=camp_hcp['T_IMS_ID'].astype(str)
            else:
                print("\nInvalid Response. Please type 1 or 2 or 3")                                     
    elif (lift_calc_indep=="y"):
        print("\nTest-Control pairs, Test Data, Control Data & Campaign reached HCPs data will be considered as per the inputs provided in 'Input Parametrs.xlsx' file")
        print("\nIs the 'Lift - Inputs' sheet in the excel file updated with all the locations (Y/N)")
        update="NA"
        while (update.lower() != "y"):
            update = input()
            if (update.lower() == "n"):
                print("\nPlease update the sheet and type Y")
            elif (update.lower() == "y"):
                print("\nLocations in 'Lift - Inputs' sheet will be used")
            else:
                print("\nInvalid Response. Please type Y or N")
        lift_path=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Lift - Inputs',index_col='input_parameter')
        tc_pair=pd.read_csv(lift_path['value']['tc_pair_inp_path'])
        test_data=pd.read_csv(lift_path['value']['test_data_path'])
        control_data=pd.read_csv(lift_path['value']['control_data_path'])
        camp_hcp=pd.read_csv(lift_path['value']['reached_ims_path'])
    
    print("\nInput Datasets are imported")
    return tc_pair,test_data,control_data,camp_hcp


### Lift calculation Interactive function
def lift_interactive():
    print("\nWould you like to run lift calculation interactively (Y/N)?")
    response="NA"
    while ((response.lower() != "y") & (response.lower() != "n")):
        response=input()
        if (response.lower() == "n"):
            interactive="n"
            print("\nThe tool will be run with the inputs provided in 'Input Parametrs.xlsx' file")
            print("\nIs the 'Lift - Interactive' sheet in the excel file updated with all the parameters (Y/N)")
            update="NA"
            while (update.lower() != "y"):
                update = input()
                if (update.lower() == "n"):
                    print("\nPlease update the sheet and type Y")
                elif (update.lower() == "y"):
                    print("\nParameters in 'Lift - Interactive' sheet will be used")
                else:
                    print("\nInvalid Response. Please type Y or N")
        elif (response.lower() == "y"):
            interactive="y"
            print("\nThe tool will be run intractively\n")
        else:
            print("\nInvalid Response. Please type Y or N")
    return interactive

### Lift calculation periods input
def lift_periods(test_data):
    test_data=test_data.copy()
    print("\nWhich metric would you like to calculate lift on (TRX / NRX)?")
    response="NA"
    while ((response.lower() != "trx") & (response.lower() != "nrx")):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['param_1']
            print(response)
        if ((response.lower() == "trx") | (response.lower() == "nrx")):
            global lift_param
            lift_param=response.upper()
            print("\nLift calculation parameter will be considered as ",lift_param)
        else:
            print("\nInvalid Response. Please type TRX / NRX")
    
    print("\nEnter months to be considered as pre period separated by a comma(,) and a space( )")
    print("Eg. 07, 08, 09, 10, 11")
    print("Where 01 represents latest month as per data")
    response = "NA"
    while (np.invert(set(response).issubset(test_data.columns))):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['columns_1']
            print(response)
        response=response.split(', ')
        response=[lift_param + "_" + x for x in response]
        if (set(response).issubset(test_data.columns)):
            pre_period=response.copy()
            print("\nColumns to be used as Pre Period")
            print(pre_period)
        else:
            print("\nInvalid Response. Please type correct month/s")
    print("\nEnter months to be considered as post period separated by a comma(,) and a space( )")
    print("Eg. 01, 02, 03, 04, 05")
    print("Where 01 represents latest month as per data")
    response = "NA"
    while (np.invert(set(response).issubset(test_data.columns))):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['columns_2']
            print(response)
        response=response.split(', ')
        response=[lift_param + "_" + x for x in response]
        if (set(response).issubset(test_data.columns)):
            post_period=response.copy()
            print("\nColumns to be used as Post Period")
            print(post_period)
        else:
            print("\nInvalid Response. Please type correct month/s")
    return pre_period,post_period

### Lift calculation Master data creation
def lift_master(tc_pair,test_data,control_data,camp_hcp):
    tc_pair=tc_pair.copy()
    test_data=test_data.copy()
    control_data=control_data.copy()
    camp_hcp=camp_hcp.copy()
    
    ### For Difference of Differences lift calculation
    cross_master_data=tc_pair.loc[(tc_pair.T_IMS_ID.isin(camp_hcp.T_IMS_ID.unique())) &
                                 ~(tc_pair.C_IMS_ID.isin(camp_hcp.T_IMS_ID.unique())),:]
    test_data.columns="T_"+test_data.columns
    control_data.columns="C_"+control_data.columns
    
    cross_master_data=pd.merge(cross_master_data,test_data,on='T_IMS_ID',how='left')
    cross_master_data=pd.merge(cross_master_data,control_data,on='C_IMS_ID',how='left')
    
    ### For ANCOVA Lift calculation
    test_data_univ=cross_master_data.loc[:,cross_master_data.columns.str.startswith('T_')]
    test_data_univ.columns=test_data_univ.columns.str[2:]
    control_data_univ=cross_master_data.loc[:,cross_master_data.columns.str.startswith('C_')]
    control_data_univ.columns=control_data_univ.columns.str[2:]
    
    test_data_univ['TC_Flag']='T'
    control_data_univ['TC_Flag']='C'
    vertical_master_data=test_data_univ.append(control_data_univ,ignore_index=True)
    
    return cross_master_data, vertical_master_data

### Lift calculation Impact
def lift_impact(cross_master_data,pre_period,post_period,vertical_master_data):
    cross_master_data=cross_master_data.copy()
    pre_period=pre_period.copy()
    post_period=post_period.copy()
    vertical_master_data=vertical_master_data.copy()
    print("\nChoose a method from below for lift calculation (1/2/3)\n")
    print("1 - Difference of Differences")
    print("2 - ANCOVA")
    print("3 - Placeholder")
    response="NA"
    while (np.invert(response in ['1','2','3'])):
        if (lift_int=="y"):
            response=input()
        else:
            response=str(lift_par['value']['param_2'])
            print(response)
        if (response=='1'):
            test_pre=cross_master_data[["T_" + x for x in pre_period]].mean(axis=1).mean()
            test_post=cross_master_data[["T_" + x for x in post_period]].mean(axis=1).mean()
            control_pre=cross_master_data[["C_" + x for x in pre_period]].mean(axis=1).mean()
            control_post=cross_master_data[["C_" + x for x in post_period]].mean(axis=1).mean()
            camp_impact=(test_post-test_pre)-(control_post-control_pre)
            camp_lift=camp_impact/control_post
            print("\nLift calculated through Difference of Differences ")
            
            ##Dummy variables for ANCOVA output if DOD is chose for calculating Lift
            emmeans=pd.DataFrame(np.nan,index=[0], columns=['0'])
            ancova_summary=pd.DataFrame(np.nan,index=[0], columns=['0'])
        elif (response=='2'):
            camp_impact,camp_lift,emmeans,ancova_summary=ancova_impact(pre_period,post_period,vertical_master_data,cross_master_data)
            print("\nLift calculated through ANCOVA")
        elif (response=='3'):
            # print("\nLift calculated through method-3")
            print("\nThis method is WIP please select 1")
            response="NA"
        else:
            print("\nInvalid Response")
    print("\nImpact of campaign on ",lift_param," is observed to be ",camp_impact,lift_param," per month per HCP")
    print("\nWith a lift of ",camp_lift*100,"%")
    return camp_impact,camp_lift,emmeans,ancova_summary

### Lift calculation through ANCOVA
def ancova_impact(pre_period,post_period,vertical_master_data,cross_master_data):
    pre_period=pre_period.copy()
    post_period=post_period.copy()
    vertical_master_data=vertical_master_data.copy()
    cross_master_data=cross_master_data.copy()
    
    ### 1-0 flag for Test & Control
    vertical_master_data['TC_Flag']=np.where(vertical_master_data['TC_Flag']=='T',1,0)
    ### Variables input for creating Pre Period and Post period metrics
    print("\nEnter metrics you wish to create pre and post variables separated by a comma(,) and a space( )")
    print("Enter prefixes of the columns already present in the data")
    print("Make sure you include lift calculation paramter (Eg. NRX)")
    print("Eg.: NRX, PDE")
    response = ["NULLVALUE"]
    while (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) > 0):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['columns_4']
            print(response)
        response=response.split(', ')
        response=[x.upper() for x in response]
        if (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) == 0):
            pre_post_var=response
        else:
            print("\nInvalid Response. Please type correct prefixes of columns")
    del response
    ### Creating Pre & Post period variables
    for var in pre_post_var:
        vertical_master_data[var+'_PRE']=vertical_master_data[[x.replace(lift_param,var) for x in pre_period]].mean(axis=1)
        vertical_master_data[var+'_POST']=vertical_master_data[[x.replace(lift_param,var) for x in post_period]].mean(axis=1)
        
        cross_master_data['T_'+var+'_PRE']=cross_master_data[['T_'+x.replace(lift_param,var) for x in pre_period]].mean(axis=1)
        cross_master_data['T_'+var+'_POST']=cross_master_data[['T_'+x.replace(lift_param,var) for x in post_period]].mean(axis=1)
        cross_master_data['C_'+var+'_PRE']=cross_master_data[['C_'+x.replace(lift_param,var) for x in pre_period]].mean(axis=1)
        cross_master_data['C_'+var+'_POST']=cross_master_data[['C_'+x.replace(lift_param,var) for x in post_period]].mean(axis=1)
        
        
    del pre_post_var
    ### Dependent variable as post lift parmater
    vertical_master_data['dep_var']=vertical_master_data[lift_param+'_POST']
    print("\nDependent variable is considered as ",lift_param+"_POST")
    print("\nDisplaying column names for reference\n")
    print(list(vertical_master_data.columns))
    print("\nEnter columns to be considered as independent variables separated by a comma(,) and a space( )")
    print("Eg. NRX_PRE, PDE_POST, SAM_POST")
    print("TC_Flag is included by default\n")
    response = "NA"
    while (np.invert(set(response).issubset(vertical_master_data.columns))):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['columns_5']
            print(response)
        response=response.split(', ')
        
        if (set(response).issubset(vertical_master_data.columns)):
            response.append('TC_Flag')
            print("\nColumns to be used as independent variables")
            print(response)
        else:
            print("\nInvalid Response. Please type correct columns")
    indep_var=response
    del response
    ### OLS regression model
    
    # defining the variables
    x = vertical_master_data[indep_var]
    y = vertical_master_data['dep_var']
     
    # adding the constant term
    x = sm.add_constant(x)
     
    # performing the regression
    # and fitting the model
    result = sm.OLS(y, x).fit()
     
    # collating results
    ancova_summary=pd.DataFrame(result.params).reset_index()
    ancova_summary.columns=['variable','coefficient']
    ancova_pvalue=pd.DataFrame(result.pvalues).reset_index()
    ancova_pvalue.columns=['variable','pvalue']
    ancova_summary=pd.merge(ancova_summary,ancova_pvalue,on='variable',how='left')
    del x,y,result,ancova_pvalue
    
    ## Pre period Lift parameter
    print("\n Choose from one of the below choices for pre period lift parameter\n")
    print("a - Average of pre period lift paramter - Control (Single fixed value)")
    print("b - Average of pre period lift paramter - Test (Single fixed value)")
    print("c - Average of pre period lift paramter - Test & Control (Single fixed value)")
    print("d - Actual pre period lift paramter of HCPs (different for different HCPs)")
    # print("e - Specific column in the data")
    # print("f - Use as per model prediction (Coefficient x pre period lift parameter)")
    print("\nEnter label number of your choice (Eg. a)")
    response="NA"
    while (np.invert(response in ['a','b','c','d','e','f'])):
        response=input()
        if (response=='a'):
            vertical_master_data['pre_lift_par']=cross_master_data['C_'+lift_param+'_PRE'].mean()
        elif (response=='b'):
            vertical_master_data['pre_lift_par']=cross_master_data['T_'+lift_param+'_PRE'].mean()
        elif (response=='c'):
            vertical_master_data['pre_lift_par']=(cross_master_data['T_'+lift_param+'_PRE'].mean() + 
                                                         cross_master_data['C_'+lift_param+'_PRE'].mean())/2
        elif (response=='d'):
            vertical_master_data['pre_lift_par']=vertical_master_data[lift_param+'_PRE']
        else:
            print("Invalid response. Please type label from the list of choices")
    print("\nMetric for Pre period lift parameter impact calculation has been created based on selected choice")
    
    # ### Getting count of Test HCPs per control
    # ## Poulating count as 1 for Test HCPs
    # control_count=cross_master_data.groupby('C_IMS_ID').agg(control_count=('T_IMS_ID','count')).reset_index().rename({'C_IMS_ID':'IMS_ID'},axis='columns')
    # vertical_master_data=pd.merge(vertical_master_data,control_count,on='IMS_ID',how='left')
    # vertical_master_data['control_count']=vertical_master_data['control_count'].fillna(1)
    # del control_count
    
    ### Impact calculation
    vertical_master_data['predicted_impact']=0
    for var in ancova_summary['variable']:
        coeff=ancova_summary.loc[ancova_summary['variable']==var,'coefficient'].reset_index(drop=True)[0]
        if var=='const':
            vertical_master_data['predicted_impact']=vertical_master_data['predicted_impact']+coeff*1
            vertical_master_data[var+'_impact']=coeff*1
        elif var==lift_param+'_PRE':
            vertical_master_data['predicted_impact']=vertical_master_data['predicted_impact']+coeff*vertical_master_data['pre_lift_par']
            vertical_master_data[var+'_impact']=coeff*vertical_master_data['pre_lift_par']
        else:
            vertical_master_data['predicted_impact']=vertical_master_data['predicted_impact']+coeff*vertical_master_data[var]
            vertical_master_data[var+'_impact']=coeff*vertical_master_data[var]
        print(var)
        
    ## Marginal means of Test & Control
    emmeans=vertical_master_data[pd.concat([pd.Series(['TC_Flag','predicted_impact']),
                                            pd.Series([x+'_impact' for x in ancova_summary['variable']])])].groupby('TC_Flag').mean().reset_index()
    # emmeans['predicted_impact']=emmeans['predicted_impact']/vertical_master_data['TC_Flag'].sum()
    # for x in ancova_summary['variable']:
    #     emmeans[x+'_impact']=emmeans[x+'_impact']/vertical_master_data['TC_Flag'].sum()
    test_impact=emmeans.loc[emmeans['TC_Flag']==1,'predicted_impact'].reset_index(drop=True)[0]
    control_impact=emmeans.loc[emmeans['TC_Flag']==0,'predicted_impact'].reset_index(drop=True)[0]
    camp_impact=(test_impact-control_impact)
    camp_lift=camp_impact/control_impact
    camp_pvalue=ancova_summary.loc[ancova_summary['variable']=='TC_Flag','pvalue'].reset_index(drop=True)[0]
    print("\nANCOVA modelling complete with pvalue of Campaign observed to be ",camp_pvalue)
    return camp_impact,camp_lift,emmeans,ancova_summary


### Lift calculation Visualization export
def lift_trends(cross_master_data,vertical_master_data):
    cross_master_data=cross_master_data.copy()
    vertical_master_data=vertical_master_data.copy()
    
    print("\nEnter oldest and latest month numbers to be used for trend charts")
    print("Eg. 12 for oldest month number, 01 for latest month number")
    print("\nEnter month number for oldest month")
    if (lift_int=="y"):
        old_month=input()
    else:
        old_month=lift_par['value']['param_3']
        print(old_month)
    
    print("\nEnter month number for latest month")
    if (lift_int=="y"):
        lat_month=input()
    else:
        lat_month=lift_par['value']['param_4']
        print(lat_month)
    
    print("\nEnter metrics you wish to compare trends b/w Test & Control separated by a comma(,) and a space( )")
    print("Enter prefixes of the columns already present in the data")
    print("Eg.: NRX, PDE")
    response = ["NULLVALUE"]
    while (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) > 0):
        if (lift_int=="y"):
            response=input()
        else:
            response=lift_par['value']['columns_3']
            print(response)
        response=response.split(', ')
        response=[x.upper() for x in response]
        if (sum(~pd.Series([sum(vertical_master_data.columns.str.startswith(x))>0 for x in response])) == 0):
            trend_data_dict={}
            for var in response:
                trend_data_1=pd.DataFrame(cross_master_data.loc[:,'T_' + var + '_' + lat_month:'T_' + var + '_' + old_month].mean(axis=0).reset_index())
                trend_data_1['index']=trend_data_1['index'].str.replace("T_"+var+"_","").astype(int)
                trend_data_1.columns=['month',"T_"+var]
                
                trend_data_2=pd.DataFrame(cross_master_data.loc[:,'C_' + var + '_' + lat_month:'C_' + var + '_' + old_month].mean(axis=0).reset_index())
                trend_data_2['index']=trend_data_2['index'].str.replace("C_"+var+"_","").astype(int)
                trend_data_2.columns=['month',"C_"+var]
                
                trend_data=pd.merge(trend_data_1,trend_data_2,on='month',how='outer').sort_values(by='month',ascending=False)
                del trend_data_1,trend_data_2
                key_name=var+'_trend'
                trend_data_dict[key_name] = copy.deepcopy(trend_data)
        else:
            print("\nInvalid Response. Please type correct prefixes of columns")
    return trend_data_dict
  
### Exporting Lift analysis data
def lift_output(trend_data_dict,emmeans,ancova_summary):
    trend_data_dict=trend_data_dict.copy()
    emmeans=emmeans.copy()
    ancova_summary=ancova_summary.copy()
    print("\nOutput file location will be considered as per the input provided in 'Input Parametrs.xlsx' file")
    print("\nIs the 'Lift - Inputs' sheet in the excel file updated with the output location (Y/N)")
    update="NA"
    while (update.lower() != "y"):
        update = input()
        if (update.lower() == "n"):
            print("\nPlease update the location and type Y")
        elif (update.lower() == "y"):
            print("\nOutput locations in 'Lift - Inputs' sheet will be used")
        else:
            print("\nInvalid Response. Please type Y or N")
    lift_path=pd.read_excel("Input/Input Parameters.xlsx",sheet_name='Lift - Inputs',index_col='input_parameter')
    with pd.ExcelWriter(lift_path['value']['output_data_path']) as writer: 
        for key in trend_data_dict:
            trend_data_dict[key].to_excel(writer, sheet_name=key,index=False)
        emmeans.to_excel(writer, sheet_name='emmeans_ANCOVA',index=False)
        ancova_summary.to_excel(writer, sheet_name='summary_ANCOVA',index=False)
        
    print("\nLift analysis trends are exported to \n",lift_path['value']['output_data_path'])
    

